In [1]:
import os
import getpass
import tenacity
import sentinelsat

### Obtain scihub credentials

In [11]:
if not ('SCIHUB_USERNAME' in os.environ and
        'SCIHUB_PASSWORD' in os.environ):
    os.environ['SCIHUB_USERNAME'] = input('Scihub Username:')
    os.environ['SCIHUB_PASSWORD'] = getpass.getpass('Scihub Password:')

In [19]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

api = SentinelAPI(os.environ['SCIHUB_USERNAME'], os.environ['SCIHUB_PASSWORD'],
                  'https://scihub.copernicus.eu/dhus')

# search by polygon, time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('ROIs/uruguay_forest.geojson'))
products = api.query(footprint,
                     date=('NOW-14DAYS', 'NOW'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30),
                     producttype='S2MSI2A')

In [20]:
products.keys()

odict_keys(['bc179759-4573-4acb-8503-4af0d2b6ebd3', 'd074fba6-22d3-4a66-9e79-863b19e5b699'])

In [21]:
# download all results from the search
@tenacity.retry(stop=tenacity.stop_after_attempt(3), wait=tenacity.wait_fixed(3600))
def download_all(*args, **kwargs):
    return api.download_all(*args, **kwargs)

downloaded, triggered, failed = download_all(products.keys())

print('Downloaded {}'.format(downloaded))
print('Triggered {}'.format(triggered))
print('Failed {}'.format(failed))

Downloading:   0%|          | 0.00/84.5M [00:00<?, ?B/s]
Downloading:   1%|          | 1.05M/84.5M [00:01<01:42, 810kB/s]
Downloading:   2%|▏         | 2.10M/84.5M [00:02<01:27, 941kB/s]
Downloading:   5%|▍         | 4.19M/84.5M [00:03<00:50, 1.59MB/s]
Downloading:   6%|▌         | 5.24M/84.5M [00:03<00:44, 1.79MB/s]
Downloading:   7%|▋         | 6.29M/84.5M [00:04<00:42, 1.83MB/s]
Downloading:   9%|▊         | 7.34M/84.5M [00:05<00:55, 1.40MB/s]
Downloading:  10%|▉         | 8.39M/84.5M [00:06<00:56, 1.34MB/s]
Downloading:  11%|█         | 9.44M/84.5M [00:06<00:59, 1.26MB/s]
Downloading:  12%|█▏        | 10.5M/84.5M [00:07<01:01, 1.21MB/s]
Downloading:  14%|█▎        | 11.5M/84.5M [00:08<01:00, 1.20MB/s]
Downloading:  15%|█▍        | 12.6M/84.5M [00:09<00:59, 1.21MB/s]
Downloading:  16%|█▌        | 13.6M/84.5M [00:10<00:58, 1.22MB/s]
Downloading:  17%|█▋        | 14.7M/84.5M [00:11<00:56, 1.23MB/s]
Downloading:  19%|█▊        | 15.7M/84.5M [00:12<00:56, 1.22MB/s]
Downloading:  20%|█▉ 

Downloaded {'d074fba6-22d3-4a66-9e79-863b19e5b699': {'id': 'd074fba6-22d3-4a66-9e79-863b19e5b699', 'title': 'S2A_MSIL2A_20210407T133221_N0300_R081_T21HXE_20210407T191121', 'size': 83874224, 'md5': 'fa96d6cae2132538b930c28e9243685c', 'date': datetime.datetime(2021, 4, 7, 13, 32, 21, 24000), 'footprint': 'POLYGON((-54.968628 -32.60836364595604,-54.967285 -32.603876635963786,-54.923157 -32.45690870619971,-54.878998 -32.31000772538808,-54.83484 -32.16316927806505,-54.79074 -32.01640460171259,-54.77945 -31.978815781479003,-54.76425 -32.605676852027074,-54.968628 -32.60836364595604))', 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('d074fba6-22d3-4a66-9e79-863b19e5b699')/$value", 'Online': True, 'Creation Date': datetime.datetime(2021, 4, 7, 22, 42, 36, 864000), 'Ingestion Date': datetime.datetime(2021, 4, 7, 22, 40, 58, 627000), 'path': './S2A_MSIL2A_20210407T133221_N0300_R081_T21HXE_20210407T191121.zip', 'downloaded_bytes': 83874224}, 'bc179759-4573-4acb-8503-4af0d2b6ebd3': {'

### Automatically obtain `True Color Image (TCI)` from downloaded package and save it as a GeoTiff

In [45]:
import calval
from calval.providers.sentinel import SceneInfo, SceneData

scene_id = next(iter(products)) 
filenames = products[scene_id]['filename'].replace('.SAFE', '.zip')
print(filenames)
# print('Uploading file : ', filenames)
scene_info = SceneInfo.from_filename(filenames)
scene_info.extract_archive()
scene_data = SceneData.from_sceneinfo(scene_info)

S2B_MSIL2A_20210412T133219_N0300_R081_T21HXE_20210412T155000.zip


/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/calval/providers/scene_info.py:29: UserWarning: /home/lddm/.local/share/calval/scenes/S2B_MSIL2A_20210412T133219_N0300_R081_T21HXE_20210412T155000.SAFE already exists, not extracting
  warnings.warn('{} already exists, not extracting'.format(output_path))


In [49]:
tci_path = scene_info.get_band_path('TCI')
geo_raster = tl.GeoRaster2.open(tci_path)

In [50]:
geo_raster

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

...

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

In [26]:
geo_raster.save('test_s2.tiff')

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho

/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/lddm/Development/forests-monitoring/.venv_37/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<autho